<a href="https://colab.research.google.com/github/ranabilal09/Reaserch-Assistent/blob/main/Reaserch_Assistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone  https://github.com/ranabilal09/Reaserch-Assistent.git

Cloning into 'Reaserch-Assistent'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [8]:
!pip install --quiet --upgrade langchain langgraph langchain-google-genai requests beautifulsoup4 python-dotenv

In [5]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('Gemini_Api_Key')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langchai_api_key')
os.environ["LANGCHAIN_PROJECT"] = 'Research-Assistent'

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from bs4 import BeautifulSoup
import requests

#prompt
Template = """ {text}

-------------------------

Using the above context , answer in short the following question:

> {question}

If the question cannot be anwsered using the text , imply summarize the text. Include all the factual information , numbers and states etc. """
summary_prompt = ChatPromptTemplate.from_template(Template)

url = "https://blog.langchain.dev/announcing-langsmith/"

def scrape_url(url):
  try:
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      page_text = soup.get_text(separator=" " , strip = True)
      return page_text
    else:
      return f"failed to retrive the websearch , request status code:{response.status_code}"
  except Exception as e:
    print(e)
    return f"Failed to retrieve the web search , An error occurred: {str(e)}"

page_content = scrape_url(url)[:10000]

Chain = summary_prompt | ChatGoogleGenerativeAI(model= "gemini-1.5-flash-8b") | StrOutputParser()

Chain.invoke({
    "question":"what is langsmith?",
    "text": page_content
})

"LangSmith is a platform designed for building and iterating on LLM-powered applications.  It's a unified system for debugging, testing, evaluating, and monitoring these applications.  It provides deep visibility into model inputs and outputs, allows for easy dataset creation and testing of prompts/chains, integrates with evaluation modules (heuristic and LLM), and facilitates monitoring of application performance, costs, and user interactions.  LangSmith is currently in closed beta.\n"